Now that we have a SQLite database with indices, page titles, and coordinate strings, let's make a database where we extract all the metadata out of those coordinate strings so it's queryable.

This should be run after the other notebook that extracts the coordinate strings.

In [33]:
import csv
import json
from wikiparse import indexer, syntax_parser as sp
import time
import os
import sqlite3
import random

In [34]:
dumps = indexer.load_dumps(build_index=False, scratch_folder='py3')
english = dumps['en']

opening E:/enwiki-20190101-pages-articles-multistream.xml/scratch\py3\index.db
current mapping 19.1 m pages

__init__ complete


In [35]:
# english.db.close()

In [36]:
c = english.cursor

Before we create the database let's get a complete list of the entries we're going to want. That is, let's look at all the coordinate strings we've extracted from each page and extract the list of keywords from there. 

In [38]:
result = c.execute('''SELECT page_num,coords,title FROM indices WHERE coords != ""
''').fetchall()
coordStrings = {item[0]:item[1] for item in result}
idx_to_title = {item[0]:item[2] for item in result}
list(coordStrings.items())[:10]

[(66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (105, 'coord|42|30|N|1|31|E|display=inline,title'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (139,
  'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown||Coord|10|36|N|172|24|E|display=inline||Coord|13|19|N|169|9|W|display=inline'),
 (140,
  'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing||Coord|30|12|N|74|7|W|name=Apollo 8 S-IC impact||Coord|31|50|N|37|17|W|name=Apollo 8 S-II impact||Coord|8|8|N|165|1|W|name=Apollo 8 estimated splashdown'),
 (161,
  'Coord|12|30|40|N|69|58|27|W|type:isle|display=title||Coord|12|31|07|N|70|02|09|W||Coord|12|31|01|N|70|02|04|W|'),
 (166, 'coord|0|N|25|W|region:ZZ_type:waterbody|display=inline,title'),
 (168, 'Coord|12|30|S|18|30|E|display=title||Coord|8|50|S|13|20|E|type:city'),
 (177,
  'Coord|55|N|115|W|type:adm1st_scal

In [39]:
len(coordStrings), type(coordStrings)

(1152376, dict)

If a page has more than one coordinate string, choose the one that's displayed at the top (`display=title`) or the first.

In [40]:
for page_num in coordStrings:
    if '||' in coordStrings[page_num]:
        pageCoordStrings = coordStrings[page_num].split('||')
        coordStrings[page_num] = pageCoordStrings[0]
        for s in pageCoordStrings:
            if "display=title" in s:
                coordStrings[page_num] = s

In [41]:
list(coordStrings.items())[:10]

[(66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (105, 'coord|42|30|N|1|31|E|display=inline,title'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (139, 'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown'),
 (140, 'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'),
 (161, 'Coord|12|30|40|N|69|58|27|W|type:isle|display=title'),
 (166, 'coord|0|N|25|W|region:ZZ_type:waterbody|display=inline,title'),
 (168, 'Coord|12|30|S|18|30|E|display=title'),
 (177,
  'Coord|55|N|115|W|type:adm1st_scale:10000000_region:CA-AB|display=title')]

For some Coord templates, there's a `note` (https://en.wikipedia.org/wiki/Template:Coord#Examples) which contains more pipes that will cut off the rest of the template. Since that tag seems to come after the other important tags, let's
ignore this problem.

In [42]:
def getKeywords(coordString, verbose=False):
    if verbose:
        print(coordString)
    keywords = {}
    rest = []
    items = coordString.split('|')
    for item in items:
        if '=' in item:
            keywords[item.split('=')[0]] = item.split('=')[1]
        elif ':' in item:
            keywords[item.split(':')[0]] = item.split(':')[1]
        else:
            rest.append(item)
#     return keywords, '|'.join(rest)
    return keywords
print(getKeywords('Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'))
print(getKeywords('Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'))
print(getKeywords('Coord|36|42|N|3|13|E|type:city'))

{'type': 'event', 'name': 'Apollo 8 landing'}
{'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'}
{'type': 'city'}


We only care about a subset of keywords, so let's make a whitelist.

In [43]:
from collections import defaultdict
keywords = defaultdict(int)
for coordString in coordStrings.values():
    for kw in getKeywords(coordString).keys():
        keywords[kw.strip()] += 1
keywords

defaultdict(int,
            {'type': 287434,
             'display': 1099820,
             'region': 432293,
             'notes': 1748,
             'name': 20998,
             'dim': 4491,
             'format': 133330,
             'source': 67662,
             'globe': 2834,
             'scale': 8045,
             'id': 217,
             'accessdate': 305,
             'entrydate': 42,
             'title': 270,
             'journal': 9,
             'volume': 9,
             'number': 1,
             'pages': 12,
             'author1': 1,
             'date': 62,
             'doi': 8,
             'bibcode': 2,
             'display-authors': 1,
             'label': 1,
             '3': 143,
             'url': 151,
             'DK_type': 1,
             'trans-title': 2,
             'language': 7,
             'publisher': 99,
             'type_landmark_region': 2,
             'region:US': 1,
             '2': 5,
             "The ''globe''&nbsp;[[File": 25,
           

In [44]:
{kw for kw,count in keywords.items() if count > 10}

{'3',
 'GNS]] coordinates adjusted using [[Google Maps]] and [http',
 'GNS]] coordinates adjusted using [[Google Maps]], and [http',
 "The ''globe''&nbsp;[[File",
 '_source',
 'access-date',
 'accessdate',
 'author',
 'date',
 'dim',
 'display',
 'entrydate',
 'first',
 'format',
 'globe',
 'id',
 'last',
 'name',
 'notes',
 'p',
 'pages',
 'publisher',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'website',
 'work',
 'year',
 '{{#expr'}

Most of these look good except for a few that are duplicates or a weird artifact from the imperfect string processing. 

In [45]:
entriesList = [
 'accessdate',
 'date',
 'dim',
 'display',
 'elevation',
 'format',
 'globe',
 'id',
 'name',
 'nosave',
 'notes',
 'publisher',
 'reason',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'work']
for e in entriesList:
    print(e, 'TEXT,')

accessdate TEXT,
date TEXT,
dim TEXT,
display TEXT,
elevation TEXT,
format TEXT,
globe TEXT,
id TEXT,
name TEXT,
nosave TEXT,
notes TEXT,
publisher TEXT,
reason TEXT,
region TEXT,
scale TEXT,
source TEXT,
title TEXT,
type TEXT,
upright TEXT,
url TEXT,
work TEXT,


It would be nice to see examples of values for the keywords so we can see how they're used.

In [46]:
from numpy.random import shuffle
def findKeywordExample(kw, count=1, debug=False):
    kws = []
    coordStringsList = list(coordStrings.values())
    shuffle(coordStringsList)
    for cs in coordStringsList:
        if f'|{kw}=' in cs:
            kws.append(cs.split(f'|{kw}=')[1].split('|')[0])
            if debug:
                print(cs)
        elif f'|{kw}:' in cs:
            kws.append(cs.split(f'|{kw}:')[1].split('|')[0])
            if debug:
                print(cs)
        if len(kws) >= count:
            break
    return kws

In [47]:
findKeywordExample('name', 10, True)

coord|42.440556|-98.148083|type:landmark|name=Ashfall Fossil Beds
coord|42|37|37|N|85|1|28|W|name=Vermontville Opera House|region:US_type:landmark
coord|76|5|30|N|109|41|59|W|region:CA-NU_type:waterbody_scale:500000|display=inline, title|name=Eldridge Bay
coord|-34.42928|172.681582|format=dms|name=Start of SH 1N|type:landmark_region:NZ|display=inline
coord|60.872|-69.323|display=inline,title|name=Eider Islands
coord|55.3057|N|117.8961|W|name=Amerada Crown GF23-11|display=inline,title
coord|75|15|N|105|00|W|region:CA-NU_type:waterbody |display=inline,title|name=Byam Channel
coord|53.59526|-2.50612|type:landmark|name=Blundell Arms Public House
coord|48|07|49.22|N|07|21|22.32|E|type:city|name=Bennwihr Gare
Coord|32.1276|N|110.4317|W|type:mountain_region:US|name=Forest Hill (peak)


['Ashfall Fossil Beds',
 'Vermontville Opera House',
 'Eldridge Bay',
 'Start of SH 1N',
 'Eider Islands',
 'Amerada Crown GF23-11',
 'Byam Channel',
 'Blundell Arms Public House',
 'Bennwihr Gare',
 'Forest Hill (peak)']

Now we can create all the columns en masse.

In [50]:
c.execute('''CREATE TABLE coords
    (coords TEXT,
    lat REAL DEFAULT 0,
    lon REAL DEFAULT 0,
    page_num INTEGER PRIMARY KEY,
    accessdate TEXT DEFAULT '',
    date TEXT DEFAULT '',
    dim TEXT DEFAULT '',
    display TEXT DEFAULT '',
    elevation TEXT DEFAULT '',
    format TEXT DEFAULT '',
    globe TEXT DEFAULT '',
    id TEXT DEFAULT '',
    name TEXT DEFAULT '',
    nosave TEXT DEFAULT '',
    notes TEXT DEFAULT '',
    publisher TEXT DEFAULT '',
    reason TEXT DEFAULT '',
    region TEXT DEFAULT '',
    scale TEXT DEFAULT '',
    source TEXT DEFAULT '',
    title TEXT DEFAULT '',
    type TEXT DEFAULT '',
    upright TEXT DEFAULT '',
    url TEXT DEFAULT '',
    work TEXT DEFAULT '')
''')

In [51]:
# c.execute('''DROP TABLE coords''')

In [59]:
def extract_lat_lon(coord_string):
    split = coord_string.split('|')
    coord_list = []
    for s in split:
        if ':' in s or '=' in s:
            break
        if 'Coord' not in s and 'LAT' not in s and 'LONG' not in s and 'coord' not in s:
            coord_list.append(s)  
    return coord_list

begin = random.randint(0, len(coordStrings)-10)
for i in range(begin, begin+10):
    print(extract_lat_lon(list(coordStrings.values())[i]))

['41.4347', 'N', '25.1328', 'E']
['48.860', '2.327']
['50.82', '-1.45']
['49.49', '0.10']
['49', '26', '02', 'N', '0', '12', '24', 'E']
['47', '31', 'N', '11', '09', 'E']
['51', '30', '44', 'N', '00', '09', '48', 'W']
['51.4988', '-0.0901']
['39', '10', 'N', '26', '20', 'E']
[]


In [60]:
def convert_to_decimal(coord_list):
#     print(' '.join(coord_list), end='\t')
    coord_list = [s.strip().lower() for s in coord_list if s.strip() != '']
    if len(coord_list) < 2:
        return [0, 0]
    if len(coord_list) == 2:
        return [float(coord_list[0]), float(coord_list[1])]
    directions = 0
    for s in coord_list:
        s = s.strip().lower()
        if s and s.strip() in 'nesw':
            directions += 1
    if directions != 2:
        raise Exception(directions, "wrong number of directions for:", coord_list)
        
    lat = []
    lon = []
    creating_lat = True
    for s in coord_list:
        s = s.strip().lower()
        if s == '':
            continue
        if creating_lat:
            if s in 'ns':
                creating_lat = False
                while len(lat) < 3:
                    lat.append(0)
                if s == 'n':
                    lat.append(1)
                else:
                    lat.append(-1)
            else:
                if ',' in s:
                    s = s.replace(',', '.')
                lat.append(float(s))
        else:
            if s in 'ew':
                while len(lon) < 3:
                    lon.append(0)
                if s == 'e':
                    lon.append(1)
                else:
                    lon.append(-1)
            else:
                if ',' in s:
                    s = s.replace(',', '.')
                lon.append(float(s))
    return [
        (lat[0] + lat[1]/60 + lat[2]/3600) * lat[3],
        (lon[0] + lon[1]/60 + lon[2]/3600) * lon[3]
    ]

count = 1000
begin = random.randint(0, len(coordStrings)-count)
for i in range(begin, begin+count):
    coord_list = extract_lat_lon(list(coordStrings.values())[i])
    try:
        result = convert_to_decimal(coord_list)
    except Exception as e:
        print(coord_list, e)
#     print(result)

In [61]:
def insertKeywordDict(page_num, coords, kws, debug=False):
    allowed_keys = [kw for kw in kws.keys() if kw in entriesList]
    allowed_vals = [kws[kw] for kw in allowed_keys]
    
    try:
        lat,lon = convert_to_decimal(extract_lat_lon(coords))
    except:
        lat = lon = 0
    keys = ['coords', 'lat', 'lon', 'page_num'] + allowed_keys
    vals = [coords, lat, lon, page_num] + allowed_vals
    
    qstring = f'INSERT INTO coords (\
        {",".join(keys)}) VALUES (\
        {",".join(["?" for item in vals])})'
    
    if debug:
        print(qstring, vals)
    c.execute(qstring, vals)
# insertKeywordDict({'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'})

Grab a drink before running the next cell, it'll take a while.

In [71]:
start = 100_000
succeeded = 0
coordStringsList = list(coordStrings.items())
for i in range(start, len(coordStringsList)):
    page_idx,coordString = coordStringsList[i]
    try:
        keywords = getKeywords(coordString)
        keywords['title'] = idx_to_title[page_idx]
        insertKeywordDict(page_idx, coordString, keywords)
        succeeded += 1
    except sqlite3.IntegrityError:
        pass
    if i % 100 ==  0:
        english.db.commit()
        print(f'{round(100*i/len(coordStrings), 2)} ({i}) succeeded: {succeeded}', end='\r')


In [72]:
# c.execute('''SELECT * FROM coords WHERE page_num > 19000000''').fetchall()

In [73]:
l = c.execute('''SELECT title FROM coords WHERE 
         lat BETWEEN 47.6 AND 47.7
        AND lon BETWEEN -122.35 AND -122.34
    ''').fetchall()
l

[('Museum of Pop Culture',),
 ('Bill & Melinda Gates Foundation',),
 ('The Art Institute of Seattle',),
 ('Space Needle',),
 ('School of Visual Concepts',),
 ('Fremont, Seattle',),
 ('City University of Seattle',),
 ('Woodland Park Zoo',),
 ('Belltown, Seattle',),
 ('Fremont Troll',),
 ('Cranium, Inc.',),
 ('Seattle Aquarium',),
 ('The Real World: Seattle',),
 ('Fremont Bridge (Seattle)',),
 ('Aurora Bridge',),
 ('Attachmate',),
 ('Woodland Park (Seattle)',),
 ('Seattle Opera',),
 ('Denny Park (Seattle)',),
 ('Victor Steinbrueck Park',),
 ('Unexpected Productions',),
 ('Memorial Stadium (Seattle)',),
 ('Bad Animals Studio',),
 ('Westlake, Seattle',),
 ('B. F. Day Elementary School',),
 ('Cutter & Buck',),
 ('Mercer Arena',),
 ('Seattle Cinerama',),
 ('Area code 206',),
 ('Waiting for the Interurban',),
 ('2006 Seattle Jewish Federation shooting',),
 ("Beth's Cafe",),
 ('Pike Place Fish Market',),
 ('SkyCity',),
 ('Antioch University Seattle',),
 ('Moore Theatre',),
 ('McLeod Residence'

How many cities do we have?

In [92]:
l = c.execute('''SELECT title,coords FROM coords WHERE 
         type == "city"
    ''').fetchall()
len(l)

7924

What's the coverage for this tag?

In [75]:
[el[0] for el in l if 'Seattle' in el[0]]

[]

In [27]:
[el[0] for el in l if 'Portland' in el[0]]

['Portland, Indiana', 'South Portland, Maine', 'Portland, Pennsylvania']

In [29]:
[el[0] for el in l if 'Brooklyn' in el[0]]

['Sea Gate, Brooklyn']

Not great.

In [98]:
l = c.execute('''SELECT title,type,lat,lon FROM coords''').fetchall()
[el for el in l if 'Seattle' == el[0]]

[('Seattle', '', 47.609722222222224, -122.33305555555555)]

That's because Seattle isn't tagged as a city.

In [99]:
[el for el in l if "Portland, Oregon" == el[0]]

[('Portland, Oregon',
  'city(568380)_region',
  45.519999999999996,
  -122.68194444444445)]

In [101]:
[el for el in l if "San Francisco" == el[0]]

[('San Francisco', '', 37.78333333333333, -122.41666666666667)]

In [102]:
len(l)

1152376

Hmm, probably too many to do TF-IDF on all of them. Can we narrow it down?

In [132]:
l = c.execute('''SELECT title,coords FROM coords WHERE 
         display == "inline,title"
    ''').fetchall()
len(l)

602882

In [133]:
[el for el in l if 'Seattle' == el[0]]

[('Seattle', 'coord|47|36|35|N|122|19|59|W|region:US-WA|display=inline,title')]

In [135]:
[el for el in l if 'Portland, Oregon' == el[0]]

[('Portland, Oregon',
  'coord|45|31|12|N|122|40|55|W|type:city(568380)_region:US-OR_source:gnis-1136645|display=inline,title')]

This narrows it down by ~50%, at least.

In [136]:
l[:50]

[('Andorra', 'coord|42|30|N|1|31|E|display=inline,title'),
 ('Atlantic Ocean',
  'coord|0|N|25|W|region:ZZ_type:waterbody|display=inline,title'),
 ('Aegean Sea',
  'coord|39|N|25|E|type:waterbody_dim:500000|display=inline,title'),
 ('Amsterdam', 'coord|52|22|N|4|54|E|region:NL|display=inline,title'),
 ('Anatolia', 'Coord|39|N|35|E|type:country|display=inline,title'),
 ('Aberdeenshire',
  'coord|57|9|3.6|N|2|7|22.8|W|type:adm2nd_region:GB|format=dms|display=inline,title'),
 ('Azincourt', 'coord|50.46|2.13|format=dms|display=inline,title'),
 ('Geography of Azerbaijan',
  'Coord|40|30|N|47|30|E|type:country_region:AZ|display=inline,title'),
 ('Adelaide',
  'coord|34|55|44|S|138|36|4|E|type:city_region:AU-SA|display=inline,title'),
 ('Athens', 'coord|37|59|02.3|N|23|43|40.1|E|format=dms|display=inline,title'),
 ('Ames, Iowa',
  'coord|42|02|05|N|93|37|12|W|region:US-IA|display=inline,title'),
 ('Abensberg', 'coord|48|48|N|11|51|E|format=dms|display=inline,title'),
 ('Park Güell',
  'Coord|

I could also open this dataset of cities that would narrow it down a lot further, but I would have to do a bunch of fuzzy name matching probably :(

In [112]:
import pandas as pd

In [116]:
index_col = ['nn', 'name', 'name_ascii', 
        'other_names', 'lat', 'lon', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'region', 'date']

In [117]:
len(index_col)

19

In [120]:
cities_df = pd.read_csv('cities15000.tsv', sep='\t', header=None, names=index_col)

In [126]:
cities_df[cities_df.f != float('nan')].sample(10)

,nn,name,alt_name,other_names,lat,lon,a,b,c,d,e,f,g,h,i,j,k,region,date
15964,1697175,Olongapo,Olongapo,"Bandaraya Olongapo,Ciudad ti Olongapo,Dakbayan...",14.82917,120.28278,P,PPLA3,PH,NaN,03,64,037107000,NaN,221178,NaN,11,Asia/Manila,2017-12-27
4879,2881485,Landshut,Landshut,"Gorad Landsgut,Landishuta,Landsgut,Landshuad,L...",48.52961,12.16179,P,PPLA2,DE,NaN,02,092,09261,09261000,60488,NaN,488,Europe/Berlin,2014-02-08
298,3855974,Esquel,Esquel,"EQS,Ehskel',Eskelis,Esquel,ai si ke er,ayskywl...",-42.91147,-71.31947,P,PPL,AR,NaN,04,26035,NaN,NaN,28486,NaN,577,America/Argentina/Catamarca,2015-04-22
14950,1028434,Quelimane,Quelimane,"Gorad Kelimaneh,Kelimane,Kelimanė,Quelimane,UE...",-17.87861,36.88833,P,PPLA,MZ,NaN,09,NaN,NaN,NaN,188964,NaN,9,Africa/Maputo,2012-01-19
4007,3682108,Garzón,Garzon,"GLJ,Garzon,Garzón",2.19593,-75.62777,P,PPL,CO,NaN,16,41298,NaN,NaN,29451,NaN,834,America/Bogota,2017-06-21
3084,1529452,Hoxtolgay,Hoxtolgay,"Chia-shih-t'o-lo-kai,Chia-shih-t’o-lo-kai,Hesh...",46.51872,86.00214,P,PPLA4,CN,NaN,13,NaN,NaN,NaN,22000,NaN,804,Asia/Urumqi,2013-06-04
16582,756868,Tomaszów Lubelski,Tomaszow Lubelski,"Liublino Tomasuvas,Liublino Tomašuvas,Tomashev...",50.44767,23.41616,P,PPLA2,PL,NaN,75,0618,061801,NaN,20261,275.0,274,Europe/Warsaw,2010-10-14
10703,1269834,Ichalkaranji,Ichalkaranji,"Icalkarandzi,Ichalkaranji,aychalkaranjy,aychl ...",16.69117,74.46054,P,PPL,IN,NaN,16,530,NaN,NaN,274383,NaN,561,Asia/Kolkata,2014-10-13
12517,248414,Kurayyimah,Kurayyimah,"Kareime,Kereime,Kuraiyima,Kuraymah,Kurayyimah,...",32.27639,35.59938,P,PPL,JO,NaN,18,NaN,NaN,NaN,17837,NaN,-183,Asia/Amman,2016-05-07
18765,65170,Baardheere,Baardheere,NaN,2.34464,42.27644,P,PPLL,SO,NaN,06,NaN,NaN,NaN,42240,NaN,97,Africa/Mogadishu,2006-01-27


Insert fuzzy name matching (if necessary)